# Advent of Code 2018
## Day 5
Polymer reactions and performance testing

In [1]:
import numpy as np
import numba

### Read in the file

We'll test a number of ways of loading in the string from the text file and converting it to a list (or array) of characters

The first one we came up with during the lunch and learn:

In [2]:
def read_input_original(filename):
    char_list = []
    with open(filename, "r") as file:
        for line in file:
            for char in line:
                char_list.append(char)
    return char_list

In [3]:
%timeit read_input_original("input.txt")

3.4 ms ± 53.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ok, benchmark performance is set. There's a fair bit of code and it's easy to follow, but it's not very pythonic and I bet I can make it faster

In [4]:
def read_input_comprehension(filename):
    with open(filename, "r") as file:
        char_list = [char for line in file for char in line]
    return char_list

In [5]:
%timeit read_input_comprehension("input.txt")

1.89 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The list comprehension is a bit less readable, since you have to work backwards, we want the character, for each line in the file, for each character in the line.
On the plus side it's pythonic and it's over twice as fast as the first implementation

In [6]:
def read_input_np(filename):
    in_str = np.array(list(np.loadtxt(filename, dtype=np.unicode_).tolist()))
    return in_str

In [7]:
%timeit read_input_np("input.txt")

5.89 ms ± 705 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


I suspect that doing the actual analysis will end up faster if I can work with numpy arrays rather than python lists.
This works, but I don't like it. It's slow, and the nested function calls are hard to read.

In [8]:
def read_input_easy_np(filename):
    with open(filename, "r") as file:
        in_str = [line for line in file][0]
    char_arr = np.array(list(in_str))
    return char_arr

In [9]:
%timeit read_input_easy_np("input.txt")

4.31 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


This is a little faster, which is interesting since it's using less numpy. It's pretty readable, and it's faster than the first numpy attempt, but it's still not as fast as even the first implementation

In [10]:
def read_input_final_np(filename):
    with open(filename, "r") as file:
        char_list = [char for line in file for char in line]
    char_arr = np.array(char_list)
    return char_arr        

In [11]:
%timeit read_input_final_np("input.txt")

4.96 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


This one's the same as the fast implementation, except I convert the list to a numpy array. It's still slow, so it seems like converting to numpy is the slow part.
It's a little slower than easy_np, but it doesn't rely on there only being one line in the file, which is nice

### Character comparison

The next thing we need to be able to do is compare two characters and determine if they react.

Do some setup code

In [12]:
# Set our fast read input as an easy name to use further
read_input = read_input_comprehension

In [13]:
# Helper function to compare results
def test_comparison(comp_func):
    char_list = read_input("input.txt")
    for i in range(1, len(char_list)):
        comp_func(char_list[i - 1], char_list[i])
    return True

Here's the method we defined at lunch

In [14]:
def compare_characters_original(char1, char2):
    if (char1.upper() == char2.upper()) & (char1.isupper() != char2.isupper()):
        return True
    return False

In [15]:
%timeit test_comparison(compare_characters_original)

21.5 ms ± 887 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


During lunch we figured there would be more characters of different case than there would be the same letters, so the comparison should be faster with the more restrictive condition first, let's test that

In [16]:
def compare_characters_diff_order(char1, char2):
    if (char1.isupper() != char2.isupper()) & (char1.upper() == char2.upper()):
        return True
    return False

In [17]:
%timeit test_comparison(compare_characters_diff_order)

21.5 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Ok, a little slower, but not material, what if we explicitly nest it?

In [18]:
def compare_characters_fast_exit(char1, char2):
    if char1.upper() != char2.upper():
        return False
    elif char1.isupper() == char2.isupper():
        return False
    return True

In [19]:
%timeit test_comparison(compare_characters_fast_exit)

17.1 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ok, that's a lot less readable, but it is a little faster. It's generally a bad idea to return early in a function. It doesn't matter too much in a small function like this, but in a larger one it can make it much harder to interpret. Not a good habit to get in, although taking 75% of the time is pretty nice. Let's see if we can do better

In [20]:
def compare_characters(char1, char2):
    if (char1.upper() == char2.upper()) & (char1 != char2):
        return True
    else:
        return False

In [21]:
%timeit test_comparison(compare_characters)

17.4 ms ± 347 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ok, that's in between. As readable as the first implementation, but a little faster. We'll go with that.

### The full algorithm

This is the part that's actually quite slow. Let's see if we can speed it up

First we make some helper functions

In [22]:
def validate_method(compression_func):
    """Read in the example text and make sure it gives the result you want"""
    eg_list = read_input("example.txt")
    result_list = compression_func(eg_list)
    result = len(result_list)
    assert result == 10
    return True

In [23]:
def run_full_method(compression_func):
    full_list = read_input("input.txt")
    answer = len(compression_func(full_list))
    return answer

Here's the original method:

In [24]:
def compress_polymer_method_1(input_list):
    polymers_removed = 1
    while polymers_removed > 0:
        polymers_removed = 0
        for i in range(len(input_list) - 1):
            if i >= len(input_list) - 1:
                break
            if compare_characters(input_list[i], input_list[i + 1]):
                input_list.pop(i)
                input_list.pop(i)
                polymers_removed += 1
    return input_list

In [25]:
validate_method(compress_polymer_method_1)

True

In [26]:
%timeit run_full_method(compress_polymer_method_1)

7.32 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Ok, that's a pretty slow speed to beat, let's try and get it faster

In [27]:
def compress_polymer_method_2(input_list):
    i = 0
    while i < len(input_list) - 1:
        if compare_characters(input_list[i], input_list[i + 1]):
            input_list.pop(i)
            input_list.pop(i)
            if i != 0:
                i -= 1
        else:
            i += 1
    return input_list

In [28]:
validate_method(compress_polymer_method_2)

True

In [29]:
%timeit run_full_method(compress_polymer_method_2)

221 ms ± 7.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Well, that's a huge speedup already. But can we go faster?

In [30]:
# Have to get a numpy array to do numpy stuff
read_input = read_input_final_np

Does doing it with a numpy array rather than a list on its own do anything for us?

In [31]:
def compress_polymer_method_3(input_list):
    i = 0
    while i < len(input_list) - 1:
        if compare_characters(input_list[i], input_list[i + 1]):
            input_list = np.delete(input_list,[i, i+1])
            if i != 0:
                i -= 1
        else:
            i += 1
    return input_list

In [32]:
validate_method(compress_polymer_method_3)

True

In [33]:
%timeit run_full_method(compress_polymer_method_3)

1.11 s ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Ooooh, counterintuitive! Numpy is slower!

[Numba](http://numba.pydata.org/) time!

In [34]:
@numba.jit(nopython=True)
def compress_polymer_method_4(input_list):
    i = 0
    while i < len(input_list) - 1:
        if (input_list[i].upper() == input_list[i + 1].upper()) & (input_list[i] != input_list[i + 1]):
            input_list = np.delete(input_list,[i, i+1])
            if i != 0:
                i -= 1
        else:
            i += 1
    return input_list

In [35]:
# validate_method(compress_polymer_method_4)

Ok, upon further investigation it looks like numba only helps with numeric work. I could do the work of converting the characters to a numeric representation, but nahhhhhh

# Final fast version

In [36]:
def compress_polymer(filename):
    with open(filename, "r") as f:
        input_str = f.read()
    output_str = ""
    for i in range(len(input_str)):
        if len(output_str) == 0:
            output_str += input_str[i]
        elif compare_characters(output_str[-1], input_str[i]):
            output_str = output_str[:-1]
        else:
            output_str += input_str[i]
    return len(output_str)

In [37]:
assert compress_polymer("example.txt") == 10

In [38]:
%timeit compress_polymer("input.txt")

34.8 ms ± 271 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
compress_polymer("input.txt")

9060